# HW 5


I _Pratul Chandra Perla_ declare that I have completed this
computer code in accordance with the UAB Academic Integrity
Code and the UAB CS Honor Code. I/We have read the UAB
Academic Integrity Code and understand that any breach of the
Code may result in severe penalties.


Student signature(s)/initials: _Perla_


Date: _April 13 2023_

## Algorithm Implementation

In [2]:
!pip install torch transformers datasets -q

In [3]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
from torch.utils.data import DataLoader

In [4]:
# Function to check and use GPU if available
def check_gpu():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        print("CUDA not available. Using CPU.")
        return torch.device("cpu")

In [5]:
# Load the IMDb dataset
dataset = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
# Use a lighter model for quicker training
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Tokenization and encoding of the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

In [8]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
# Ensure that labels are properly named and included
tokenized_datasets = tokenized_datasets.map(lambda examples: {'labels': examples['label']}, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
# Function to calculate accuracy
def compute_accuracy(predictions, labels):
    preds_flat = predictions.argmax(dim=1).flatten()
    labels_flat = labels.flatten()
    correct_pred = (preds_flat == labels_flat).cpu().numpy()
    return np.mean(correct_pred)

In [11]:
# Device configuration
device = check_gpu()
device

device(type='cuda')

In [31]:
model.to(device)
# Reduced dataset size for faster training and validation
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))  # smaller subset
val_dataset = tokenized_datasets["test"].select(range(200))  # smaller subset
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

In [32]:
# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_loader) * 3  # number of epochs
lr_scheduler = get_scheduler("linear",
                             optimizer=optimizer,
                             num_warmup_steps=0,
                             num_training_steps=num_training_steps,)

In [33]:
# Training and validation loop
model.train()
for epoch in range(3):
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        if loss is not None:
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
        else:
            print("Loss is None, check the inputs and model configuration.")

    # Validation phase
    model.eval()
    total_eval_accuracy = 0
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        total_eval_accuracy += compute_accuracy(logits, batch["labels"])

    avg_val_accuracy = total_eval_accuracy / len(val_loader)
    print(f'Epoch {epoch+1} | Accuracy: {avg_val_accuracy:.4f}%')
    model.train()

Epoch 1 | Accuracy: 0.7212%
Epoch 2 | Accuracy: 0.8317%
Epoch 3 | Accuracy: 0.8365%


## Performance Improvement

In [34]:
# Check and use GPU if available, else use CPU
def check_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        print("CUDA not available. Using CPU.")
        return torch.device("cpu")

In [35]:
# Load and prepare the IMDb dataset
def prepare_dataset():
    dataset = load_dataset("imdb")
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    tokenized_datasets = tokenized_datasets.map(lambda examples: {'labels': examples['label']}, batched=True)
    tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    return tokenized_datasets

In [36]:
# Function to calculate accuracy
def compute_accuracy(predictions, labels):
    preds_flat = predictions.argmax(dim=1).flatten()
    labels_flat = labels.flatten()
    correct_pred = (preds_flat == labels_flat).cpu().numpy()
    return np.mean(correct_pred)

In [37]:
# Set up model and data loaders
def setup_model_and_loaders(device, train_dataset, val_dataset):
    model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
    model.to(device)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

    return model, train_loader, val_loader

In [42]:
# Training and validation
def train_and_validate(model, train_loader, val_loader, device):
    optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
    num_training_steps = len(train_loader) * 5  # 3 epochs
    lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    model.train()
    for epoch in range(5):
        for batch in train_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        # Validation phase
        model.eval()
        total_eval_accuracy = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(**batch)
                logits = outputs.logits
                total_eval_accuracy += compute_accuracy(logits, batch["labels"])

        avg_val_accuracy = total_eval_accuracy / len(val_loader)
        print(f'Epoch {epoch+1} | Accuracy: {avg_val_accuracy:.4f}')
        model.train()
    return model

In [43]:
def main():
    device = check_device()
    tokenized_datasets = prepare_dataset()
    train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))  # Reduced for quicker training
    val_dataset = tokenized_datasets["test"].select(range(200))  # Reduced for quicker validation
    model, train_loader, val_loader = setup_model_and_loaders(device, train_dataset, val_dataset)
    trained_model = train_and_validate(model, train_loader, val_loader, device)

if __name__ == "__main__":
    main()

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 | Accuracy: 0.7356
Epoch 2 | Accuracy: 0.9519
Epoch 3 | Accuracy: 0.8702
Epoch 4 | Accuracy: 0.8462
Epoch 5 | Accuracy: 0.8558


## Converting file into HTML file

In [44]:
!pip install nbconvert -q

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
%cd '/content/drive/MyDrive/Colab_Notebooks'

/content/drive/MyDrive/Colab_Notebooks


In [48]:
!jupyter nbconvert --to html CS655_HW5.ipynb

[NbConvertApp] Converting notebook CS655_HW5.ipynb to html
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbconvert", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/jupyter_core/application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 423, in start
    self.convert_notebooks()
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 597, in convert_notebooks
    self.convert_single_notebook(notebook_filename)
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 560, in convert_single_notebook
    output, resources = self.export_single_notebook(
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 488, in e